In [1]:
from langchain_groq import ChatGroq

from langchain_core.prompts import PromptTemplate

/Users/fardeenkhan/coding/Agentic Ai/venev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing import TypedDict, List

class AgentState(TypedDict):
    query: str
    db_choice: List[str]
    documents: List[str]
    answer: str

In [ ]:


llm = ChatGroq(model="llama-3.1-8b-instant",)

router_prompt = PromptTemplate(
    template="""
You are a routing agent.

Available databases:
- pinecone: production knowledge base
- astradb: real-time & transactional data
- chroma: research papers & PDFs

User query: {query}

Return a Python list of database names to query.
Example: ["pinecone", "chroma"]
""",
    input_variables=["query"]
)

def router_agent(state: AgentState):
    response = llm.invoke(router_prompt.format(query=state["query"]))
    return {"db_choice": eval(response.content)}

In [4]:
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()

python-dotenv could not parse statement starting at line 2


True

In [6]:
groq_api=os.getenv("groq_api_key")

In [8]:
from dotenv import load_dotenv,dotenv_values

In [52]:
groq_api_key=os.getenv("groq_api_key")
os.environ["groq_api_key"]=groq_api_key


In [53]:
pinecone_api=os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_API_KEY"]=pinecone_api


In [54]:
ASTRA_DB_API_TOKENs=os.getenv("ASTRA_DB_API_ENDPOINT")
os.environ["ASTRA_DB_API_ENDPOINT"]=ASTRA_DB_API_TOKENs


In [55]:
ASTRA_DB_API_ENDPOINT=os.getenv("ASTRA_DB_API_ENDPOINT")
os.environ["ASTRA_DB_API_ENDPOINT"]=ASTRA_DB_API_ENDPOINT


In [14]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader,PyPDFLoader,CSVLoader

In [15]:
headphone_loader = DirectoryLoader(
    path="dataset",   # folder containing the file
    glob="*.csv",     # pattern to match CSV files
    loader_cls=CSVLoader
)
headphone= headphone_loader.load()

In [16]:
book_loader = DirectoryLoader(
    path="dataset/Amazon_Books_Scraping",   # folder containing the file
    glob="*.csv",     # pattern to match CSV files
    loader_cls=CSVLoader
)
book= book_loader.load()

In [17]:
fashion_loader= DirectoryLoader(
    path="dataset",
    glob="**/*.pdf",
    loader_cls=PyPDFLoader
)

fashion= fashion_loader.load()

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

headphone_chunks= splitter.split_documents(headphone)
books_chunks = splitter.split_documents(book)
fashion_chunks = splitter.split_documents(fashion)

In [19]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",
    encode_kwargs={"normalize_embeddings": True}
)

In [20]:
from langchain_pinecone import Pinecone

In [21]:
 
from pinecone import Pinecone,ServerlessSpec

In [23]:
from langchain_community.vectorstores import Pinecone
import pinecone



In [24]:

from langchain_community.vectorstores import Chroma

research_store = Chroma(
    collection_name="fashiom",
    embedding_function=embeddings,
    persist_directory="./chroma_research"
)

research_store.add_documents(fashion_chunks)

FASHION_retriever = research_store.as_retriever(k=5)

/var/folders/r9/z_0p1s8d181c9f73p9bdkf940000gn/T/ipykernel_3847/1818343488.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  research_store = Chroma(
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 2


In [ ]:
from langchain_astradb import AstraDBVectorStore

headphone_vectorstore = AstraDBVectorStore(
    embedding=embeddings,
    collection_name="headphone_chunk",
    api_endpoint="ASTRA_DB_API_ENDPOINT",
    token="ASTRA_DB_API_TOKENs"
)

headphone_vectorstore.add_documents(books_chunks)

BOOK_retriever = headphone_vectorstore.as_retriever(k=5)

In [26]:
from langchain_pinecone import PineconeVectorStore

In [27]:
embedding_model=HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

In [28]:
from pinecone import Pinecone,ServerlessSpec
pine_cone_api=pinecone_api

pc=Pinecone(api_key=pinecone_api)

In [30]:
index_name="amazon"

In [31]:


if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")

    )

In [32]:
index=pc.Index(index_name)

In [33]:
index

In [56]:
pinecone_api=os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_API_KEY"]=pinecone_api


In [35]:
docsearch=PineconeVectorStore.from_documents(
    documents=headphone_chunks,
    embedding=embedding_model,
    index_name=index_name
)

In [36]:
headphone_retriver=docsearch.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [37]:
FASHION_retriever.invoke("what is fashion")

[Document(metadata={'page': 27, 'creationdate': '2017-07-07T13:25:04+05:30', 'moddate': '2017-07-07T13:35:22+05:30', 'title': '01_Fashion design_20june2017.cdr', 'total_pages': 123, 'author': 'priyanka', 'source': 'dataset/17-fashion_design_7july2017_1.pdf', 'producer': 'Corel PDF Engine Version 18.0.0.448', 'creator': 'Adobe Acrobat 8.0 Combine Files', 'page_label': '28'}, page_content='balancing a figure.\n3. Fashionfigure- Sketched and stylized model of human figure.\n4. Vertical balance line- A centre line in a block or stick figure which divides the left and right side of \nfigure equally from head to base. It is also called centre front in a front view of a fashion figure.\n5. StickFigure- A drawing of fashion figure made out of lines without forming any block.\n6. Fleshing- Adaptation of a block/robo or stick figure into a fashion figure.\nINTRODUCTION\nSketching gives a shape to the imagination and creativity of a designer. Fashion Illustration is the \ncommunication of fashion

In [38]:
from typing import TypedDict, List

class AgentState(TypedDict):
    query: str
    datasets: List[str]
    documents: List[str]
    answer: str

In [39]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

llm = ChatGroq(model="llama-3.1-8b-instant",api_key=groq_api_key)

router_prompt = PromptTemplate(
    template="""
You are a dataset routing agent.

Available datasets:
- fashion → about fashion ,clothes,shirt ,pants
- headphone→ headphone,earphone ,flipkart data
-  books→ title ,author ,number of perdon read it

User query: {query}

Return a Python list from:
["fashion", "music", books"]
""",
    input_variables=["query"]
)

def router_agent(state: AgentState):
    result = llm.invoke(router_prompt.format(query=state["query"]))
    return {"datasets": eval(result.content)}

In [40]:
def FASHION_agent(state: AgentState):
    if "fashion" not in state["datasets"]:
        return {}
    docs = FASHION_retriever.invoke(state["query"])
    return {"documents": [d.page_content for d in docs]}

In [41]:
def HEADPHONE_agent(state: AgentState):
    if "research" not in state["datasets"]:
        return {}
    docs = headphone_retriver.invoke(state["query"])
    return {"documents": [d.page_content for d in docs]}

In [42]:
def BOOKS_agent(state: AgentState):
    if "realtime" not in state["datasets"]:
        return {}
    docs = BOOK_retriever.invoke(state["query"])
    return {"documents": [d.page_content for d in docs]}

In [43]:
def merge_agent(state: AgentState):
    unique_docs = list(dict.fromkeys(state["documents"]))
    return {"documents": unique_docs}

In [44]:
answer_prompt = PromptTemplate(
    template="""
Answer the question using ONLY the context below.

Context:
{documents}

Question:
{query}
""",
    input_variables=["documents", "query"]
)

def answer_agent(state: AgentState):
    response = llm.invoke(
        answer_prompt.format(
            documents="\n".join(state["documents"]),
            query=state["query"]
        )
    )
    return {"answer": response.content}

In [47]:
from langgraph.graph import StateGraph

In [48]:
graph = StateGraph(AgentState)

graph.add_node("router", router_agent)
graph.add_node("fashion", FASHION_agent)
graph.add_node("books", BOOKS_agent)
graph.add_node("headphone", HEADPHONE_agent)
graph.add_node("merge", merge_agent)
graph.add_node("answer", answer_agent)

graph.set_entry_point("router")

graph.add_conditional_edges(
    "router",
    lambda state: state["route"],
    {
        "fashion": "fashion",
        "books": "books",
        "headphone": "headphone",
    }
)

graph.add_edge("fashion", "merge")
graph.add_edge("books", "merge")
graph.add_edge("headphone", "merge")

graph.add_edge("merge", "answer")

app = graph.compile()

In [49]:
print(app.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	router(router)
	fashion(fashion)
	books(books)
	headphone(headphone)
	merge(merge)
	answer(answer)
	__end__([<p>__end__</p>]):::last
	__start__ --> router;
	books --> merge;
	fashion --> merge;
	headphone --> merge;
	merge --> answer;
	router -.-> books;
	router -.-> fashion;
	router -.-> headphone;
	answer --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [50]:
app.get_graph().draw_mermaid_png(output_file_path="langgraph_agents.png")

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01|\x00\x00\x02\x13\x08\x02\x00\x00\x00\x8b\xcf0\x8f\x00\x00\x10\x00IDATx\x9c\xec\x9d\x07@\x14G\x17\xc7g\xaf\xd1;\x82\x8a *bCE\xc5n\xb0`\x8b\xc6(jl\xb1\xc4Xc\xd7\x18c\x89\x1a5\x1a\xbb\xc6\x96hb,Q,\x9f=\xf6\xa8\xb1Dc\x17\xbb\x89(\x16\x10P\x8at\xb8\xb6\xdf\xbb[8\x0e\x04\x02\xc2\xed\xed\xdd\xbc_\xf8\xee\x9b\x9d\xdd\x9d;wf\xff\xfb\xe6\xbd\xd9\x19\t\xcb\xb2\x04A\x10\x84/$\x04A\x10\x84GPt\x10\x04\xe1\x15\x14\x1d\x04Ax\x05E\x07A\x10^A\xd1A\x10\x84WPt\x10\x04\xe1\x15\x14\x1d\xa1\x93\x12/\x0f\xbd\xf0\xf6MDff:\xabV\xb1\ny\xce\x10\x07F\xc4\x10\xed\x90\x07\x11\xc3\xa8\xd5\xacH\xa4\xf9\x84|\xb1\x98Q\xa9r%t\x88DD\xad\x86\x13\tC\x18\x96\xcd\x1a/\xc10\x9a\xb4H\xcc\xa8U\x90&\xbaQ\x14\x8c\xe6\x04\x86\xd5\x96\xa9+\x9cKs\xe7r\xa5ir4\xe7\xe6|\x91L\n-\x8b\xb1\xb4\x14\x95\xf1\xb4\xa8\x1bho\xe7dI\x10$\x1b\x06\xc7\xe9\x08\x13y\xa6r\xff\x9aWqQr\x10\x02\xa9\x8c\xc8\xac\xc42\x0b\x11\x88\x852S+\x06\x9cX\x88\x88JMD\x90\x10\x13V\xa5\xd9d\xb5\x12\xc0H\x18V\xa9U\n1\x81\xd

In [ ]:
result = app.invoke({
    "query": "what is price of headphone"
})

print(result["answer"])

In [57]:
Model_Cache = {}

In [58]:

import time
def cached_model(query):
    start_time = time.time()
    if Model_Cache.get(query):
        print("***CACHE HIT***")
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        return Model_Cache.get(query)
    else:
        print("***CACHE MISS – EXECUTING MODEL***")
        start_time = time.time()
        response = llm.invoke(query)
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        Model_Cache[query] = response
        return response

In [ ]:
query="WHAT IS THE BESTT HEADPHONE "
response = cached_model(query)
print(response)